- How to identity contact between buses?
- 100m range --> references
    - Trajectory Improves Data Delivery in Vehicular Networks = 300m
    - Classificando Comportamentos Sociais em Redes Veiculares = 100 to 150m, janelas de agregação --> 5,15,30,60
    - Explorando Dados Urbanos: um estudo usando viagens deT ́axi da cidade de S ̃ao Francisco 100m e 15 min
    - An Alternative Approach to Mobility Analysis inVehicular Ad Hoc Network - 100m
    - CARACTERIZAÇÃO DA MOBILIDADE VEICULAR E O SEU IMPACTO NASREDES VEICULARES TOLERANTES A ATRASOS E DESCONEXÕES - 100m
    - nsights on Metropolitan-Scale VehicularMobility from a Networking Perspectiv - 100m e grafo
    - Generation and Analysis of a Large-Scale UrbanVehicular Mobility Dataset 100m
- Temporal Graph --> references procurar depois


In [2]:
# Getting the minute of the date_avl, and getting the graph_id
# The register will be modeled in a temporal graph where there is a graph for a moment. 
# This graph is composed by nodes which are buses, 
# and edges are connections between buses (nodes) if they are closer from each other less 100 meters

# A graph in the moment t is composed by register in a interval of 15 seconds. Interval examples:
# 10:00 - 10:00:14
# 10:00:15 - 10:00:29
# There will be a graph for each one of the interval above.
# To define which interval the trace belong:
# 1- We get the column hour_avl  (previous calculated) from the date of the register. Ex: 10:30 --> hour_avl = 10
# 2- We get the column minute_avl the minute from the date of the register. Ex: 10:35 --> minute_avl = 35
# 3- We calculate the second_avl of the date_avl ex:
# 3- Calculate minute_avl/15, then we round to floor the division, and we have the interval in that hour. 
# Ex: 10:35, floor(35/15) = 2, so the interval is the second in the hour 10
# 4- We concate the hour_avl, and the interval (partition) = graph_id. So, we have graph_id
# Example of the intervals and graph id
# 10:00 - 10:14:59 - interval 0 - graph_id = 10-0
# 10:15 - 10:29:59 - interval 1 - graph_id = 10-1
# 10:30 - 10:44:59 - interval 2 - graph_id = 10-2
# 10:45 - 10:59:59 - interval 3 - graph_id = 10-3


import pyspark.sql.functions as F

days_to_analyze = [1,4,12,17,20]

for day in days_to_analyze:
    traces = spark.read.parquet("s3://mobility-traces-sp/processed-data-avl-date/13-speed-calculation-filtered-columns/MO_1510"+str(day)+"/")
    df_graph_id = traces\
        .withColumn('minute_avl',F.minute(F.col("dt_avl")))\
        .withColumn("second_avl",F.second("dt_avl"))\
        .withColumn("graph_id",F.concat(F.col("hour_avl"),F.lit("-"),F.floor((F.col("minute_avl")*60+F.col("second_avl"))/15)))

    df_graph_id.repartition("graph_id").write.parquet("s3://mobility-traces-sp/processed-data-avl-date/14-speed-and-graph-id-15-seconds/MO_1510"+str(day)+"/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import pyspark.sql.functions as f

new_df = spark.read.parquet("s3://mobility-traces-sp/processed-data-avl-date/14-speed-and-graph-id/MO_1510"+str(1)+"/")
new_df = new_df.repartition(150)
#new_df = df.withColumn('graph_id_minutes',f.concat(F.col("hour_avl"),f.lit("-"),f.col("minute_avl"),f.lit("-"),f.col("region")))
new_df = new_df.withColumn('graph_id_novo',f.concat(f.col("hour_avl"),f.lit("-"),f.floor(f.col("minute_avl")/5),f.lit("-"),f.col("region")))

df2 = new_df

# new_df.groupby("region","graph_id_minutes").agg(F.count("id_avl").alias("teste")).select(F.max("teste")).show()
new_df.alias('df1').join(df2.alias("df2"),on=["graph_id_novo"],how="outer")\
    .select(
        f.col("df1.id_avl").alias("id_avl_1"),
        f.col("df1.line_id").alias("line_1"),
        f.col("df1.latitude").alias("latitude_1"),
        f.col("df1.longitude").alias("longitude_1"),
        f.col("df2.id_avl").alias("id_avl_2"),
        f.col("df2.line_id").alias("line_2"),
        f.col("df2.latitude").alias("latitude_2"),
        f.col("df2.longitude").alias("longitude_2"),
        f.col("graph_id_novo").alias("graph_id"),
    
).write.parquet("s3://mobility-traces-sp/teste_minute_MO_15101_5min/")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [1]:
x

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1610034250435_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'x' is not defined
Traceback (most recent call last):
NameError: name 'x' is not defined



In [25]:
traces = spark.read.parquet("s3://mobility-traces-sp/processed-data-avl-date/14-speed-and-graph-id-each-15/MO_1510"+str(1)+"/")

import pyspark.sql.functions as F

# traces = traces.repartition(40)
# traces.alias('l').join(traces.alias('r'), on='graph_id')\
#     .select(f.col("l.dt_avl").alias("dt_avl"), f.col('r.latitude').alias('longitude2')).where('r.id_avl != l.id_avl').write.parquet("s3://mobility-traces-sp/bebebeeb1213/MO_15101/")
# sql.functions.broadcas


# traces.alias('l').join(traces.alias('r'),on=["graph_id"],how="outer").select(f.col("r.dt_avl").alias("dt_avl")).write.parquet("s3://mobility-traces-sp/bebebeeb1213/MO_151013232313123/")



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
y = x.repartition(100)
y.alias('l').join(y.alias('r'),on=["novo_id"],how="outer").select(F.col("r.dt_avl").alias("dt_avl")).write.parquet("s3://mobility-traces-sp/bebeb999eeb1213/MO_34343sdasdasd23424232/")



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…